## YΣ19 Artificial Intelligence II
# Homework 2

### Iglezou Myrto - 111520170038

# Question 5

In [1]:
import pandas as pd 
from pandas import DataFrame, read_csv

In [2]:
# import gdown
# url = 'https://drive.google.com/file/d/1dTIWNpjlrnTQBIQtaGOh0jCRYZiAQO79/view?usp=sharing'
# path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
# output = "twitterData.csv"
# gdown.download(path, output, quiet=False)
# twitterData = pd.read_csv("/content/twitterData.csv")
# df = pd.DataFrame(data=twitterData)
# df.head(5)

path = r"C:\Users\myrto\Documents\GitHub\AI2-project2\SentimentTweets.csv"
file = pd.read_csv(path,index_col=False)
df = pd.DataFrame(data=file)
df.head(5)

,Unnamed: 0,target,id,date,flag,user,text
0,680949,0,2249621587,Fri Jun 19 22:41:08 PDT 2009,NO_QUERY,sukumarpant,#brokenpromises...
1,406741,0,2059003515,Sat Jun 06 16:03:21 PDT 2009,NO_QUERY,MTMSparrow,David Carradine so sad. Thai's law not sure i...
2,1337108,4,2017466467,Wed Jun 03 08:26:14 PDT 2009,NO_QUERY,itsmemcee,A @ 415 B @ 425. Tell your bro i say congrats!
3,1560887,4,2186457254,Mon Jun 15 18:52:04 PDT 2009,NO_QUERY,jdfreivald,@littlefluffycat Indeed.
4,1466295,4,2064458395,Sun Jun 07 06:19:20 PDT 2009,NO_QUERY,CrazyHan,Completed Race 4 Life in 58mins with girlies f...


In [57]:
X = df['text']
y = df['target']

## Data pre-processing

### Removing the scpecial characters

In [58]:
import re

def RemoveSpecialCharacters(x):
    x = re.sub(r'[^\w]+', " ", x)
    return x

In [59]:
X = X.apply(lambda x: RemoveSpecialCharacters(x) )

### Lemmatize

In [60]:
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

X = X.apply(lambda x: WordNetLemmatizer().lemmatize(x))

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\myrto\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Removing the upercase

In [61]:
X = X.apply(lambda x: x.lower())

### Create a list o all words

In [62]:
words = []
for row in X:
    sentence = row.split()
    for word in sentence:
        words.append(word)
        
words.append("<pad")

### Create the vocabulary

In [63]:
from collections import Counter

counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
word_2_id = {w:i for i,w in enumerate(vocab,1)}

### Encode the words

In [64]:
def convert_tokens_to_inds(sentence, word_2_id):
    return [word_2_id.get(t) for t in sentence]

In [65]:
X = X.apply(lambda x: x.split())

In [66]:
X = X.apply(lambda x: convert_tokens_to_inds(x, word_2_id))

In [67]:
X

0                                                   [194640]
1          [860, 4915, 18, 114, 3405, 12, 1548, 26, 202, ...
2          [4, 32137, 333, 50419, 276, 46, 973, 1, 181, 630]
3                                              [34359, 1064]
4          [3234, 1366, 159, 186, 10, 194641, 23, 5710, 5...
                                 ...                        
1279995    [138603, 141, 5444, 7, 541, 86, 1, 44, 16, 34,...
1279996                                       [9, 424, 3006]
1279997    [186222, 18, 126, 2, 288, 63, 46, 322, 16, 12,...
1279998    [589806, 8, 19, 193, 53, 8, 146, 19389, 9, 4, ...
1279999                [1, 142, 333, 5072, 1, 153, 2684, 12]
Name: text, Length: 1280000, dtype: object

### Check for empty rows

In [68]:
rows = Counter([len(x) for x in X])
print("Zero-length sentences: {}".format(rows[0]))

Zero-length sentences: 0


### Padding for windows.

Find the maximum sentence lenght and make every sentence that long, by adding 0 on the right and left.

In [69]:
def pad_sentence_for_window(sentence, max_row, pad_token=0):
    padding = max_row - len(sentence)
    
    left = (int(padding/2) if padding%2 ==0 else int(padding/2 + 1) )
    right = int(padding/2)
    return [pad_token]*left + sentence + [pad_token]*right

In [70]:
max_row = max(rows)

X = X.apply(lambda x: pad_sentence_for_window(x,max_row))

### Transform data to a 2D array

In [71]:
import numpy as np
X = np.vstack(X.values)

In [72]:
y = y.values.tolist()
y = np.array(y)

### Split data

In [73]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 4, stratify = y)

### Batching sentences together with a DataLoader

In [74]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# Tensor datasets
train_data = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
test_data = TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test))

# dataloaders
batch_size = 32

# SHUFFLE your data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)